# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/forward_qt.pickle','rb') as f:
    forward_qt = pickle.load(f)

forward_model = load_model('./models/forward_nn.h5')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
forward_df = premier_df[premier_df['포지션']=='공격수']

In [5]:
# 0골인 선수 보정
forward_df['골'] = forward_df['골']+1

# Shoot Accuracy Weighted Goal
forward_df['SAWG'] = (forward_df['경기당 유효 슈팅/90'] / forward_df['경기당 슈팅/90'])*forward_df['골']

In [6]:
# 0도움인 선수 보정

forward_df['도움'] = forward_df['도움']+1

# Pass Accuracy Weighted Assist And Keypass
forward_df['PAWAAK'] = (forward_df['경기당 패스 성공/90'] / forward_df['경기당 패스 시도/90'])*forward_df['도움']+forward_df['키패스/90']

In [7]:
forward_df['태클 시도/90'] = forward_df['태클 성공/90']/forward_df['태클 성공률']

In [8]:
features = ('PAWAAK','SAWG','기회 창출/90','드리블 성공/90','반칙 수/90','크로스 성공률','태클 시도/90','파울 획득')

In [9]:
forward_df.loc[:,features] = forward_qt.transform(forward_df.loc[:,features])

In [10]:
forward_df = forward_df.reset_index(drop=True)

In [11]:
independent_val = ('파울 획득', 'SAWG', '태클 시도/90', '기회 창출/90', '크로스 성공률', 'PAWAAK', '드리블 성공/90')

In [12]:
forward_df['예측PPG']=forward_model.predict(tf.constant(forward_df.loc[:,independent_val].values))

In [13]:
forward_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,SAWG,PAWAAK,태클 시도/90,예측PPG
37,Gabriel Martinelli,2021-2022,공격수,아스날,잉글랜드 프리미어 리그,5,3,0,13,7,...,1.74,4.70,0.008793,10.16,5.31,1.64,1.0,1.0,0.077359,1.676599
44,Demarai Gray,2021-2022,공격수,에버튼,잉글랜드 프리미어 리그,5,3,0,13,10,...,1.31,2.85,0.008695,5.71,2.69,1.63,1.0,1.0,0.074734,1.675767
10,Diogo Jota,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,8,4,0,18,7,...,1.81,3.07,0.009213,6.95,5.59,1.74,1.0,1.0,0.074239,1.675255
9,Sadio Mané,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,14,5,0,25,11,...,1.39,2.77,0.009239,6.04,5.14,1.91,1.0,1.0,0.064886,1.674032
60,Christian Pulisic,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,6,6,0,26,6,...,1.90,2.80,0.009417,4.37,6.49,2.00,1.0,1.0,0.064150,1.673892


In [14]:
forward_min = np.min(forward_df['예측PPG'].tolist())
forward_min

1.588832974433899

In [15]:
forward_df['WAR'] = forward_df['예측PPG'].map(lambda x : x-forward_min+1)

In [16]:
forward_war = forward_df.sort_values(by='WAR',ascending=False).copy()
forward_war = forward_war.reset_index(drop=True)
forward_war.head(50)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,SAWG,PAWAAK,태클 시도/90,예측PPG,WAR
0,Gabriel Martinelli,2021-2022,공격수,아스날,잉글랜드 프리미어 리그,5,3,0,13,7,...,4.70,0.008793,10.16,5.31,1.64,1.0,1.000000,0.077359,1.676599,1.087766
1,Demarai Gray,2021-2022,공격수,에버튼,잉글랜드 프리미어 리그,5,3,0,13,10,...,2.85,0.008695,5.71,2.69,1.63,1.0,1.000000,0.074734,1.675767,1.086934
2,Diogo Jota,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,8,4,0,18,7,...,3.07,0.009213,6.95,5.59,1.74,1.0,1.000000,0.074239,1.675255,1.086422
3,Sadio Mané,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,14,5,0,25,11,...,2.77,0.009239,6.04,5.14,1.91,1.0,1.000000,0.064886,1.674032,1.085199
4,Christian Pulisic,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,6,6,0,26,6,...,2.80,0.009417,4.37,6.49,2.00,1.0,1.000000,0.064150,1.673892,1.085060
5,Harry Kane,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,26,13,0,25,16,...,0.72,0.008309,7.34,0.95,1.82,1.0,1.000000,0.053678,1.673864,1.085032
6,Phil Foden,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,9,6,0,21,2,...,2.45,0.008954,3.09,4.57,2.50,1.0,1.000000,0.064493,1.673816,1.084983
7,Raheem Sterling,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,11,7,0,28,10,...,2.53,0.009494,4.00,4.94,1.88,1.0,1.000000,0.064152,1.673631,1.084798
8,Nathan Tella,2021-2022,공격수,사우스햄튼,잉글랜드 프리미어 리그,6,3,0,8,7,...,2.80,0.008836,5.75,2.27,1.00,1.0,1.000000,0.064351,1.673599,1.084766
9,Allan Saint-Maximin,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,5,7,0,22,6,...,2.57,0.009099,3.50,3.80,1.76,1.0,1.000000,0.064175,1.673486,1.084653


In [17]:
forward_df.query(f'이름 in ["Phil Foden"]')[['이름','WAR']]

,이름,WAR
18,Phil Foden,1.084983


In [18]:
forward_war.loc[:,features] = forward_qt.inverse_transform(forward_war.loc[:,features])

In [19]:
forward_war[['이름','시즌','골', '도움','WAR']].sort_values(by='WAR',ascending=False).head(40)

,이름,시즌,골,도움,WAR
0,Gabriel Martinelli,2021-2022,5,3,1.087766
1,Demarai Gray,2021-2022,5,3,1.086934
2,Diogo Jota,2021-2022,8,4,1.086422
3,Sadio Mané,2021-2022,14,5,1.085199
4,Christian Pulisic,2021-2022,6,6,1.085060
5,Harry Kane,2021-2022,26,13,1.085032
6,Phil Foden,2021-2022,9,6,1.084983
7,Raheem Sterling,2021-2022,11,7,1.084798
8,Nathan Tella,2021-2022,6,3,1.084766
9,Allan Saint-Maximin,2021-2022,5,7,1.084653
